In [2]:
import torch
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchvision.models import resnet18

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision
import pickle
import torch
import os
import copy
import torch.nn as nn
import seaborn as sns
from torch.utils.data import DataLoader
#from utils.dataset import CustomImageDataset, CustomImageDataset_Adv
from torchvision.transforms import transforms

In [4]:
path = "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f/MT_Baseline_1f77b0596b5392db3af2aff2a02444c8/Checkpoints/model_best.pth.tar"
loc='cpu'
x = torch.load(path, map_location=loc)

In [3]:
# Random Kernel groups
class ShardedKernels(nn.Module):
    def __init__(self, num_quadrants):
        super().__init__()
        self.num_quadrants = num_quadrants

    def generate_quad_coords(self, kernel_size):
        step_size = max(1, kernel_size // 2)
        cords_list = []
        for r_start in range(0, kernel_size, step_size):
            for c_start in range(0, kernel_size, step_size):
                cords_list.append(((r_start, r_start + step_size), (c_start, c_start + step_size)))
        return cords_list
    
    def forward(self, conv_filters):       
        conv_filters_filtered = torch.ones(size=(conv_filters.shape[0], conv_filters.shape[1] // self.num_quadrants, conv_filters.shape[2], conv_filters.shape[3])).to(device=conv_filters.device)
        quad_cords = self.generate_quad_coords(conv_filters.shape[-1])
        kernel_depth_step = conv_filters.shape[1] // self.num_quadrants
        k_depth_idx = 0
        for r_cords, c_cords in quad_cords:
            conv_filters_filtered[:, :, r_cords[0]:r_cords[1], c_cords[0]:c_cords[1]] = conv_filters[:, k_depth_idx:k_depth_idx + kernel_depth_step, r_cords[0]:r_cords[1], c_cords[0]:c_cords[1]]
            k_depth_idx+= kernel_depth_step
        
        conv_filters = conv_filters_filtered
        return conv_filters


In [4]:
conv_filters = torch.randn(size=(16, 16, 1, 1))
div_factor = 4

In [5]:
sk = ShardedKernels(4)
x =sk(conv_filters)

In [44]:
x.shape

torch.Size([16, 4, 1, 1])

In [37]:
def generate_quad_coords(kernel_size):
    step_size = max(1, kernel_size // 2)
    cords_list = []
    for r_start in range(0, kernel_size, step_size):
        for c_start in range(0, kernel_size, step_size):
            cords_list.append(((r_start, r_start + step_size), (c_start, c_start + step_size)))
    return cords_list

In [38]:
generate_quad_coords(1)

[((0, 1), (0, 1))]

In [5]:
conv_filters_filtered = torch.ones(size=(conv_filters.shape[0], conv_filters.shape[1] // div_factor, conv_filters.shape[2], conv_filters.shape[3]))
quad_cords = generate_quad_coords(8)
kernel_depth_step = conv_filters.shape[1] // div_factor
k_depth_idx = 0
for r_cords, c_cords in quad_cords:

    conv_filters_filtered[:, :, r_cords[0]:r_cords[1], c_cords[0]:c_cords[1]] = conv_filters[:, k_depth_idx:k_depth_idx + kernel_depth_step, r_cords[0]:r_cords[1], c_cords[0]:c_cords[1]]
    k_depth_idx+= kernel_depth_step

In [6]:
conv_filters[0, 96, :, :]

tensor([[ 1.1402e+00, -4.2928e-01,  2.8091e-01, -1.0190e+00, -5.9424e-01,
          4.5974e-01,  2.6081e-01,  6.7590e-01],
        [ 4.7654e-01, -5.4996e-01, -3.3806e+00, -8.3316e-01,  6.4199e-01,
         -1.2206e+00, -2.3750e-01,  3.6996e-01],
        [ 3.1071e-03,  1.3704e+00,  1.4806e+00, -2.0582e-01, -9.4039e-01,
          7.5699e-01,  2.7783e-02, -8.2270e-01],
        [-1.2132e+00,  6.1465e-01, -1.1997e+00, -8.8267e-01,  1.0682e+00,
         -4.9999e-01, -5.7061e-01, -9.7227e-01],
        [ 5.9675e-03,  5.3239e-01,  1.4067e+00, -1.3155e+00,  6.4387e-01,
          6.1400e-01, -1.2567e-01,  6.3331e-01],
        [-6.5907e-01,  1.6734e+00, -1.0897e-01,  1.0050e+00, -6.7881e-01,
          1.3858e-01, -8.6043e-01, -7.4899e-01],
        [ 1.7202e+00, -1.2103e+00, -8.6344e-01, -6.3969e-01, -1.0775e+00,
          1.5935e+00,  1.2569e-01, -1.0253e-01],
        [-1.8721e-01,  4.8032e-01,  1.6802e+00, -2.6866e-01, -8.1354e-01,
         -1.1330e+00, -3.0445e-02, -5.7410e-01]])

In [7]:
conv_filters_filtered[0, 0, :, :]

tensor([[-1.9296,  0.3216,  1.4403,  0.3847, -2.1354, -0.2196,  0.4370, -2.1858],
        [-0.2682, -0.9838, -1.0259,  0.8769,  0.6713, -0.6556, -0.0418,  0.5478],
        [ 0.0592,  2.7598, -0.6658,  0.1691, -0.8939,  0.7662, -0.4438,  0.9105],
        [ 0.3674,  0.0668, -1.1247,  0.9042, -1.4086,  0.5988,  0.1539, -0.1045],
        [-2.0813, -0.7407, -0.4134,  0.8676,  0.6439,  0.6140, -0.1257,  0.6333],
        [ 1.4898,  0.4647,  0.4583,  0.3006, -0.6788,  0.1386, -0.8604, -0.7490],
        [-0.4112, -0.9134,  0.7305, -0.8308, -1.0775,  1.5935,  0.1257, -0.1025],
        [ 0.3185, -0.9737,  1.4716, -0.4050, -0.8135, -1.1330, -0.0304, -0.5741]])

[((0, 4), (0, 4)), ((0, 4), (4, 8)), ((4, 8), (0, 4)), ((4, 8), (4, 8))]

In [7]:
div_factor = 2

group_indices = torch.randint(size=(conv_filters.shape[0], conv_filters.shape[1] // div_factor, conv_filters.shape[2],
                                             conv_filters.shape[3]), high = div_factor, low=0)
offset_indices = torch.arange(start=0, end =conv_filters.shape[1], step=div_factor )
offset_indices = offset_indices.reshape(-1, 1 ,1)
indices = torch.add(group_indices, offset_indices).to(device=conv_filters.device)

In [8]:
group_indices.shape

torch.Size([16, 64, 8, 8])

In [22]:
indices.shape

torch.Size([16, 64, 8, 8])

In [29]:
div_factor = 2
# Gather first quad


In [30]:
div_factor = 2
group_indices = torch.randint(size=(conv_filters.shape[0], conv_filters.shape[1] // div_factor, conv_filters.shape[2],
                                        conv_filters.shape[3]), high = div_factor, low=0)
offset_indices = torch.arange(start=0, end =conv_filters.shape[1], step= div_factor)
#offset_indices = offset_indices.reshape(-1, 1 ,1)
#indices = torch.add(group_indices, offset_indices).to(device=conv_filters.device)
#conv_filters = torch.gather(conv_filters, 1, indices)

In [31]:
group_indices.shape

torch.Size([16, 64, 8, 8])

In [26]:
offset_indices = torch.arange(start=0, end =conv_filters.shape[1], step= div_factor )

In [27]:
offset_indices.shape

torch.Size([64])

In [7]:
32*4

128

In [2]:
adv_dataset_path = "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f/MT_Baseline_814e4c261078e7110bf94f724cd2187e/Adversarial_Datasets/CW_adv_samples_2500_test_detector-type-Regular_integrated-False_c-1.0_d-0.0_eps-0.034.pickle"

with open(adv_dataset_path, 'rb') as adv_set:
    adv_samples = pickle.load(adv_set)



transform_adv = transforms.Compose([
    ])
valset = CustomImageDataset_Adv(adv_samples, transform=transform_adv)

val_loader = torch.utils.data.DataLoader(valset, batch_size=8,
                                            shuffle=False, num_workers=1)

In [3]:
class RandResNetFeatureExtractor(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        # Get a resnet18 backbone
        m = model
        return_nodes = {
            # node_name: user-specified key for output dict
            'x' : 'x',
            'conv1': 'conv1',
            'layer1.0.conv1': 'layer1.0.conv1',
            'layer1.0.conv2': 'layer1.0.conv2',
            'layer1.1.conv1': 'layer1.1.conv1',
            'layer1.1.conv2': 'layer1.1.conv2',
            'layer2.0.conv1': 'layer2.0.conv1',
            'layer2.0.conv2': 'layer2.0.conv2',
            'layer2.1.conv1': 'layer2.1.conv1',
            'layer2.1.conv2': 'layer2.1.conv2',
            'layer3.0.conv1': 'layer3.0.conv1',
            'layer3.0.conv2': 'layer3.0.conv2',
            'layer3.1.conv1': 'layer3.1.conv1',
            'layer3.1.conv2': 'layer3.1.conv2',
            'layer4.0.conv1': 'layer4.0.conv1',
            'layer4.0.conv2': 'layer4.0.conv2',
            'layer4.1.conv1': 'layer4.1.conv1',
            'layer4.1.conv2': 'layer4.1.conv2',
            'fc' : 'fc'
      
        }
        self.body = create_feature_extractor(
            m, return_nodes=return_nodes)
    def forward(self, x):
        x = self.body(x)
        return x



In [4]:

from utils.resnet_rand_tracer_1 import resnet18 as resnet18_1,  SparsifyKernelGroups as SparsifyKernelGroups_1 
sparseblock_1 = SparsifyKernelGroups_1

from utils.resnet_rand_tracer_2 import resnet18 as resnet18_2,  SparsifyKernelGroups as SparsifyKernelGroups_2
sparseblock_2 = SparsifyKernelGroups_2
scale_factor=4


In [5]:
model_1 = resnet18_1(sparsefilter=sparseblock_1,scale_factor=scale_factor)                
model_1.fc = nn.Linear(in_features=512, out_features=10, bias=True)

ckpt_path = "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f/MT_Baseline_814e4c261078e7110bf94f724cd2187e/Checkpoints/model_best.pth.tar"
loc = 'cpu'
checkpoint = torch.load(ckpt_path, map_location=loc)
model_1.load_state_dict(checkpoint['state_dict'])

model_1 = RandResNetFeatureExtractor(model_1)


model_2 = resnet18_2(sparsefilter=sparseblock_2,scale_factor=scale_factor)                
model_2.fc = nn.Linear(in_features=512, out_features=10, bias=True)

ckpt_path = "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f/MT_Baseline_814e4c261078e7110bf94f724cd2187e/Checkpoints/model_best.pth.tar"
loc = 'cpu'
checkpoint = torch.load(ckpt_path, map_location=loc)
model_2.load_state_dict(checkpoint['state_dict'])

model_2 = RandResNetFeatureExtractor(model_2)


In [6]:
torch.manual_seed(131)
np.random.seed(131)

torch.set_printoptions(precision=10)

In [7]:
for x, y in val_loader:
    images = x
    break

In [8]:
pred_1 = model_1(images)
pred_2 = model_2(images)

In [9]:
pred_1['layer2.0.conv2'][0, 0, :, :]

tensor([[ 0.0244943742,  0.1756346375,  0.2234019637,  0.0828140825],
        [-0.0387842506,  0.0129092969,  0.0315281153, -0.1097311974],
        [ 0.0705896765, -0.0361112505, -0.0218053516, -0.0906050354],
        [-0.1606753021, -0.2752042115, -0.1349949539, -0.0648586378]],
       grad_fn=<SliceBackward0>)

In [10]:
pred_2['layer2.0.conv2'][0, 0, :, :]

tensor([[ 0.0180419460,  0.3182912767,  0.1522731036,  0.0494774953],
        [ 0.0247102417,  0.1215718761,  0.1180688888, -0.0719897375],
        [ 0.1559844762,  0.0093886144,  0.0524684116, -0.0205788258],
        [-0.1441754699, -0.3059452772, -0.1860212982, -0.1545179784]],
       grad_fn=<SliceBackward0>)

In [11]:
class ResNetFeatureExtractor(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        # Get a resnet18 backbone
        m = model
        return_nodes = {
            # node_name: user-specified key for output dict
            'layer1.0.conv1': 'layer1a.0.conv1',
            'layer1.0.conv2': 'layer1.0.conv2',
            'layer1.1.conv1': 'layer1.1.conv1',
            'layer1.1.conv2': 'layer1.1.conv2',
            'layer2.0.conv1': 'layer2.0.conv1',
            'layer2.0.conv2': 'layer2.0.conv2',
            'layer2.1.conv1': 'layer2.1.conv1',
            'layer2.1.conv2': 'layer2.1.conv2',
            'layer3.0.conv1': 'layer3.0.conv1',
            'layer3.0.conv2': 'layer3.0.conv2',
            'layer3.1.conv1': 'layer3.1.conv1',
            'layer3.1.conv2': 'layer3.1.conv2',
            'layer4.0.conv1': 'layer4.0.conv1',
            'layer4.0.conv2': 'layer4.0.conv2',
            'layer4.1.conv1': 'layer4.1.conv1',
            'layer4.1.conv2': 'layer4.1.conv2'
        }
        self.body = create_feature_extractor(
            m, return_nodes=return_nodes)
    def forward(self, x):
        x = self.body(x)
        return x



In [ ]:
model = resnet18()
model = ResNetFeatureExtractor(model)

images = torch.randn(size=(8, 3, 32, 32))
features = model(images)
img = features['layer1.0.conv2'].detach().numpy()
img = img[0]
#img = img[:, :, 0]
img = img.swapaxes(0,1)
img = img.swapaxes(1,2)
img = img[:, :, 0]


In [ ]:
def plot_side_by_side(image, save_path=None, titles=None):
    """
    Plots three images side by side.

    Args:
        image1 (np.ndarray): The first image.
        image2 (np.ndarray): The second image.
        image3 (np.ndarray): The third image.
        titles (list, optional): A list of titles for the images.

    Returns:
        None
    """
    fig, axes = plt.subplots(1, 1, figsize=(10, 5))
    #axes[1].set_title(titles[1])
    axes.imshow(image)
    axes.axis('off')
    
    plt.tight_layout()

In [ ]:
plot_side_by_side(img)

In [ ]:

from utils.resnet_rand import resnet18, SparsifyFiltersLayer, SparsifyKernelGroups
sparseblock = SparsifyKernelGroups


In [ ]:
model_rand = resnet18()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision
import pickle
import torch
import os
import copy
import torch.nn as nn
import seaborn as sns


In [ ]:
import pickle

In [ ]:
path = "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f/MT_Baseline_43528ed29912dd1a3825f30bea8e4ba8/"


stuff = "Adversarial_Datasets/CW_adv_samples_512_test_detector-type-Regular_integrated-False_c-1.0_d-0.0_eps-0.002.pickle"

In [ ]:
all_base_paths = ["/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f/MT_Baseline_43528ed29912dd1a3825f30bea8e4ba8/"]

In [ ]:
og_num_samples = 512
    
for eps_idx, base_path in enumerate(all_base_paths):
    detector='Regular'
    num_samples = 512
    c_base=1.0
    d_base=0.0
    all_flipped_indices = []
    lst_theoretical_eps_per_channel = []
    lst_mse_loss_per_channel = []
    lst_mse_loss_per_pixel = []
    lst_mse_max_per_pixel = []
    lst_mse_loss_per_image = []

    prev_flipped_indices = set()
    eps = 0.2
    images_cw_base = f"Adversarial_Datasets/CW_adv_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
    images_benign = f"Benign_Datasets/CW_benign_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
    predictions_base = f"Predictions/Model/CW_type-adversarial_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
    filtered_indices_base = f"Predictions/Perturbed_Samples/CW_benign_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pt"
    images_cw_base = os.path.join(base_path, images_cw_base)
    images_benign = os.path.join(base_path, images_benign)
    predictions_base_path = os.path.join(base_path, predictions_base)
    filtered_indices_base_path = os.path.join(base_path, filtered_indices_base)

    # Filter by common indices
    filtered_indices_base = torch.load(filtered_indices_base_path)
    filtered_indices_base = [x.item() for x in filtered_indices_base]
    curr_flipped_indices = set(filtered_indices_base)
    curr_unique_flipped_indices = curr_flipped_indices.difference(prev_flipped_indices)    
    prev_flipped_indices.update(curr_unique_flipped_indices)
    curr_unique_flipped_indices = list(curr_unique_flipped_indices)
    all_flipped_indices.append(filtered_indices_base)

    with open(images_cw_base, 'rb') as in_file:
        images_cw_base = pickle.load(in_file)
        images_cw_base[0] = images_cw_base[0][curr_unique_flipped_indices]
        images_cw_base[1] = images_cw_base[1][curr_unique_flipped_indices]
        
    with open(images_benign, 'rb') as in_file:
        images_benign = pickle.load(in_file)
        images_benign[0] = images_benign[0][curr_unique_flipped_indices]
        images_benign[1] = images_benign[1][curr_unique_flipped_indices]

    with open(predictions_base_path, 'rb') as in_file:
        predictions = pickle.load(in_file)
        true_labels_base = np.asarray(predictions['true_labels'])[curr_unique_flipped_indices]
        pred_labels_base = np.asarray(predictions['pred_labels'])[curr_unique_flipped_indices]

    #save_path = os.path.join(base_path, 'Visualizations')
    

    for idx in range(20):
        benign = images_benign[0][idx]
        cw_base = images_cw_base[0][idx]

        benign = benign.swapaxes(0,1)
        benign = benign.swapaxes(1,2)

        cw_base = cw_base.swapaxes(0,1)
        cw_base = cw_base.swapaxes(1,2)


        title_benign = f'GT:{true_labels_base[idx]}'
        title_base = f'Cls:{pred_labels_base[idx]}'
        titles = [title_benign, title_base]
        plot_side_by_side(benign, cw_base,  idx, titles=titles)

In [ ]:
og_num_samples = 512
    
for eps_idx, base_path in enumerate(all_base_paths):
    detector='Regular'
    num_samples = 512
    c_base=1.0
    d_base=0.0
    all_flipped_indices = []
    lst_theoretical_eps_per_channel = []
    lst_mse_loss_per_channel = []
    lst_mse_loss_per_pixel = []
    lst_mse_max_per_pixel = []
    lst_mse_loss_per_image = []

    prev_flipped_indices = set()
    eps = 0.2
    images_cw_base = f"Adversarial_Datasets/CW_adv_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
    images_benign = f"Benign_Datasets/CW_benign_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
    predictions_base = f"Predictions/Model/CW_type-adversarial_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
    filtered_indices_base = f"Predictions/Perturbed_Samples/CW_benign_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pt"
    images_cw_base = os.path.join(base_path, images_cw_base)
    images_benign = os.path.join(base_path, images_benign)
    predictions_base_path = os.path.join(base_path, predictions_base)
    filtered_indices_base_path = os.path.join(base_path, filtered_indices_base)


    with open(images_benign, 'rb') as in_file:
        images_benign = pickle.load(in_file)
        images_benign[0] = images_benign[0][curr_unique_flipped_indices]
        images_benign[1] = images_benign[1][curr_unique_flipped_indices]


In [ ]:
images_benign

In [2]:
weights = torch.randn(size=(64, 3, 3))

In [8]:
base_gaussian = torch.randn_like(weights)

In [11]:
base_gaussian[0].mean()

tensor(-0.3168)

In [21]:
weights = torch.randn(size=(10000,))
mu, sigma = torch.tensor(10), torch.tensor(20)

In [27]:
weights

tensor([-1.0727, -0.8496,  0.8868,  ...,  0.4626, -0.4955,  0.0988])

In [29]:
np.sqrt(0.05)

0.22360679774997896

In [22]:
pert_weights = torch.sqrt(sigma)*weights + mu 

In [24]:
pert_weights.mean()

tensor(9.9655)

In [26]:
pert_weights.std().square()


tensor(20.0134)